# Assignment 5

Build CNN model for sentiment analysis (binary classification) of IMDB Reviews (https://www.kaggle.com/utathya/imdb-review-dataset). You can use data with label="unsup" for pretraining of embeddings. Here you are forbidden to use test dataset for pretraining of embeddings.
Your quality metric is accuracy score on test dataset. Look at "type" column for train/test split.
You can use pretrained embeddings from external sources.
You have to provide data for trials with different hyperparameter values.

You have to beat following baselines:

[3 points] acc = 0.75

[5 points] acc = 0.8

[8 points] acc = 0.9

[2 points] for using unsupervised data

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer() 
token = RegexpTokenizer('\w+')
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=7000)

In [3]:
data = pd.read_csv('imdb_master.csv', index_col=0, encoding='iso-8859-1')
data[:7]

,type,review,label,file
0,test,Once again Mr. Costner has dragged out a movie...,neg,0_2.txt
1,test,This is an example of why the majority of acti...,neg,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",neg,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,neg,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,neg,10003_3.txt
5,test,"A funny thing happened to me while watching ""M...",neg,10004_2.txt
6,test,This German horror film has to be one of the w...,neg,10005_2.txt


In [4]:
print('Размер датасета: ', len(data))
print('Кол-во неразмеченных примеров: ', len(data.loc[data["label"] == "unsup"]))

Размер датасета:  100000
Кол-во неразмеченных примеров:  50000


In [5]:
#Разделим датасет на test и train

test = data.loc[data["type"] == "test"]
train = data.loc[(data["type"] == "train") & (data["label"] != "unsup")]
print('Размер test dataset: ', len(test))
print('Размер train dataset: ', len(train))

Размер test dataset:  25000
Размер train dataset:  25000


In [6]:
def tokenize(text):
    return token.tokenize(text)


def lemmatize(texts):
    arr = []
    texts = [text.lower() for text in texts]
    for text in texts:
        words = [lemmatizer.lemmatize(word) for word in tokenize(text) if word not in stops]
        arr.append(' '.join(words))
    return arr

In [8]:
train_txt = list(train["review"])
test_txt = list(test["review"])

In [9]:
train_txt = lemmatize(train_txt)
test_txt = lemmatize(test_txt)

In [10]:
train.insert(2, "review_lem", train_txt) 

In [11]:
test['label'] = test['label'].map({'neg': 0, 'pos': 1})
train['label'] = train['label'].map({'neg': 0, 'pos': 1})

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
train.head()

,type,review,review_lem,label,file
25000,train,Story of a man who has unnatural feelings for ...,story man unnatural feeling pig start opening ...,0,0_3.txt
25001,train,Airport '77 starts as a brand new luxury 747 p...,airport 77 start brand new luxury 747 plane lo...,0,10000_4.txt
25002,train,This film lacked something I couldn't put my f...,film lacked something put finger first charism...,0,10001_4.txt
25003,train,"Sorry everyone,,, I know this is supposed to b...",sorry everyone know supposed art film wow hand...,0,10002_1.txt
25004,train,When I was little my parents took me along to ...,little parent took along theater see interior ...,0,10003_1.txt


In [13]:
test.insert(2, 'review_lem', test_txt)
test.head()

,type,review,review_lem,label,file
0,test,Once again Mr. Costner has dragged out a movie...,mr costner dragged movie far longer necessary ...,0,0_2.txt
1,test,This is an example of why the majority of acti...,example majority action film generic boring re...,0,10000_4.txt
2,test,"First of all I hate those moronic rappers, who...",first hate moronic rapper could nt act gun pre...,0,10001_1.txt
3,test,Not even the Beatles could write songs everyon...,even beatles could write song everyone liked a...,0,10002_3.txt
4,test,Brass pictures (movies is not a fitting word f...,brass picture movie fitting word really somewh...,0,10003_3.txt


In [28]:
def model(train_data, bins):
    
    model = Sequential([
    Embedding(7000, 256),
    Bidirectional(LSTM(100, return_sequences=True)),
    GlobalMaxPool1D(),
    Dense(32, activation = 'relu'),
    Dropout(0.1),
    Dense(1, activation = 'sigmoid'),
    ])
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    tokenizer.fit_on_texts(train_data)
    X_test = pad_sequences(tokenizer.texts_to_sequences(train_data), maxlen=200)
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_data), maxlen=200)
    y_train = bins
    
    model.fit(X_train, y_train, batch_size=250, epochs=3, validation_split=0.25)
    
    return model, X_test

In [29]:
model, X_test = model(train['review_lem'], train['label'])

Train on 18750 samples, validate on 6250 samples
Epoch 1/3
18750/18750 [==============================] - 643s 34ms/step - loss: 0.6338 - acc: 0.6666 - val_loss: 0.7340 - val_acc: 0.3322
Epoch 2/3
18750/18750 [==============================] - 673s 36ms/step - loss: 0.5227 - acc: 0.7357 - val_loss: 0.6378 - val_acc: 0.6730
Epoch 3/3
18750/18750 [==============================] - 735s 39ms/step - loss: 0.4046 - acc: 0.8190 - val_loss: 0.6087 - val_acc: 0.7032


In [30]:
y_pred = [int(x[0] + 0.5) for x in model.predict(X_test)]

In [36]:
print('Accuracy score: ', accuracy_score(test['label'], y_pred))

Accuracy score:  0.82928
